<a href="https://colab.research.google.com/github/P2Enjoy/ia-web3.fr/blob/main/SpeedWatch_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install all requirements (run once)

# System
import locale, os
locale.getpreferredencoding = lambda: "UTF-8"
!pip install --upgrade transformers==4.31.0 torch accelerate==0.21.0 bitsandbytes==0.41.0 tensorflow==2.13.0 onnxruntime-gpu==1.15.1
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Application for ponctuation
!pip install deepmultilingualpunctuation
!pip install scikit-learn
!pip install nltk

!pip install pytorch-crf==0.7.2 youtube_transcript_api==0.6.1 langchain sentence_transformers

# Youtube processing

In [68]:
#@title Put the link of the video to summarize:
video_link = "https://www.youtube.com/watch?v=9JRl-3mN4eo&pp=ygUIYXJpYW5lIDU%3D" #@param {type:"string"}

In [79]:
#@title Extract the youtube video transcript

def YTVideoToText(video_link):
  # installing & importing libraries
  from youtube_transcript_api import YouTubeTranscriptApi

  # fetching video transcript
  video_id = video_link.split("=")[1]
  transcript = YouTubeTranscriptApi.get_transcript(video_id)

  # iterating throughout and adding all text together (recovering ponctuations)
  flat_transcript = ""
  for tu in transcript:
      flat_transcript += ' ' + tu['text']

  return flat_transcript

video_full_text = YTVideoToText(video_link)

In [ ]:
#@title Extract tyhe video transcript for all youtube video
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

def YTVideoToText(video_link):
    # Extract video ID from the video_link
    parsed_url = urlparse(video_link)
    video_id = None
    if parsed_url.netloc == 'www.youtube.com':
        video_id = parse_qs(parsed_url.query).get('v', [None])[0]
    elif parsed_url.netloc == 'youtu.be':
        video_id = parsed_url.path.split('/')[1]

    if not video_id:
        raise ValueError("Invalid YouTube video link or missing video ID")

    # Fetching video transcript
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
    except Exception as e:
        raise ValueError("Error fetching transcript: " + str(e))

    # Iterating throughout and adding all text together (recovering punctuations)
    flat_transcript = ""
    for tu in transcript:
        flat_transcript += ' ' + tu['text']

    return flat_transcript

try:
    video_full_text = YTVideoToText(video_link)
    print(video_full_text)
except ValueError as ve:
    print(ve)


In [70]:
#@title Restore the ponctuation

from deepmultilingualpunctuation import PunctuationModel

model = PunctuationModel()
video_restored_text = model.restore_punctuation(video_full_text)

# Summarization

In [71]:
#@title Load the model into memory
# Load model directly
from transformers import AutoTokenizer, pipeline

model_id = "facebook/bart-large-cnn" #@param {type:"string"}
tokenizer = AutoTokenizer.from_pretrained(model_id)
#summarizeryt = pipeline('summarization', model= model_id, device=0)
summarizeryt = pipeline('summarization', model=model_id, device=-1)  # Utiliser le CPU (-1) au lieu du GPU (0)


In [72]:
#@title Join sentences in paragraphs of less the specified token size
summarize_token_width = 500 #@param {type:"slider", min:250, max:2048, step:250}

from langchain.text_splitter import CharacterTextSplitter
splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, separator=f'\. ', chunk_size=summarize_token_width, chunk_overlap=0
)

In [73]:
#@title Extrac the summary
def do_sum(sentences):
  # summarize text
  summarized_text = ''
  for sentence in sentences:
      out = summarizeryt(sentence, min_length=30, do_sample=False)
      out = out[0]
      out = out['summary_text']
      summarized_text += f'.\n{out}'

  # returning summary
  return summarized_text;

video_summarized_text = do_sum(splitter.split_text(video_restored_text))

In [ ]:
#@title Show the summarized text of the video
video_summarized_text

In [ ]:
#@title reformulation du resumé
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet
import random
import string

# Télécharger le modèle 'punkt' et les données WordNet si nécessaire
nltk.download('punkt')
nltk.download('wordnet')

# Fonction pour obtenir les synonymes d'un mot en utilisant la lemmatisation
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

# Fonction pour reformuler le texte en utilisant NLTK
def reformuler_texte(texte):
    # Tokenization des phrases
    phrases = sent_tokenize(texte)

    # Réformulation des phrases
    phrases_reformulees = []
    for phrase in phrases:
        # Tokenization des mots dans la phrase
        mots = word_tokenize(phrase)

        # Réformulation de chaque mot en utilisant un synonyme aléatoire
        mots_reformules = []
        for mot in mots:
            # Vérifier si le mot est un mot de ponctuation
            if mot in string.punctuation:
                mots_reformules.append(mot)  # Conserver les caractères de ponctuation inchangés
            else:
                synonyms = get_synonyms(mot)
                if synonyms:
                    mot_reformule = random.choice(synonyms)
                else:
                    mot_reformule = mot  # Conserver le mot inchangé s'il n'y a pas de synonyme
                mots_reformules.append(mot_reformule)

        # Reconstitution de la phrase réformulée
        phrase_reformulee = ' '.join(mots_reformules)
        phrases_reformulees.append(phrase_reformulee)

    # Concaténer les phrases pour former le texte réformulé
    texte_reformule = ' '.join(phrases_reformulees)

    return texte_reformule

# Réformulation du résumé original
resume_reformule = reformuler_texte(video_summarized_text)
print(resume_reformule)
